<center>
<h2>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h2>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2020</h3>
<h3>Predicción de la Calidad de Servicio</h3>
<h3>Introducción al Aprendizaje Automático</h3>
</center>
</left>
<h4>Mentor: Martín Hunziker</h4>

[Link Mentoria](https://sites.google.com/view/mentorias2020-diplodatos/ciencia-de-datos-aplicada-en-la-distribuci%C3%B3n-de-energ%C3%ADa-el%C3%A9ctrica?authuser=0).

</left>
</left>

### Introducción

En la siguiente notebook, se presentará la consigna a seguir para el tercer práctico de la materia Introducción al Aprendizaje Automático y la segunda parte de la creación de variables. 

Como referencia para el análisis geográfico utilizaremos la notebook 2_0_Intro_Variables_georefrenciadas de Ramiro Caro.


In [1]:
# Importación de las librerías necesarias
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.get_cachedir()
import matplotlib.pyplot as plt
%matplotlib inline

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString
import contextily as ctx

import networkx as nx

import seaborn as sns
import sklearn as skl

from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, Perceptron, Ridge
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, mean_squared_error, classification_report, roc_curve, auc
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from scipy.spatial import cKDTree
np.random.seed(0)  # Para mayor determinismo

In [2]:
data_path_csv = './Dataset/group_ucbt_untrd_untrs_dist_v2.csv'
data_path = './Dataset/ENF_6612_2018-12-31_M10_20190529-1610.gdb'

In [3]:
ucbt_untrd_untrs = pd.read_csv(data_path_csv) #Cargamos el dataset del práctico 2
ucbt_untrd_untrs.drop(columns=['Unnamed: 0'], axis=1,
                          inplace=True)
ssmt = gpd.read_file(data_path, driver='FileGDB', layer='SSDMT')
untrd = gpd.read_file(data_path, driver='FileGDB', layer='UNTRD')

## Consigna para la creación de Features

### I. Features Geográficas

Continuando con lo iniciado en los análisis previos, vamos a trabajar con la generación de Features a partir de grafos. 


## Consigna para Introducción al Aprendizaje Automático

### II. Preprocesamiento

#### 1. Obtención del Dataset

Trabajaremos con el Dataset aumentado con las variables que obtuvimos en el práctico 2 y en la creación de Features del presente práctico..


#### 2. Normalización de Atributos

Es posible que sea necesario normalizar las features de nuestro dataset, dado que muchos de los algoritmos de clasificación supervisada lo requieren. ¿En qué casos tendrá que implementarse normalización?

Aplicar al datasets la normalización de atributos que consideren adecuada.

#### 3. Mezca Aleatória y División en Train/Test

Finalmente, están en condiciones de **dividir el dataset en Train y Test**, utilizando para este último conjunto un 20% de los datos disponibles. Previo a esta división, es recomendable que mezclen los datos aleatoriamente.
De este modo, deberán obtener cuatro conjuntos de datos, para cada uno de los datasets: ```X_train```, ```X_test```, ```y_train``` y ```y_test```.

### III. Aplicación de Modelos de Clasificación

Una vez finalizada la etapa de preprocesamiento, se propone implementar diferentes modelos de clasificación, utilizando la librería Scikit-Learn:

1. Perceptron. Utilizar el método Stochastic Gradient Descent.
2. K Nearest Neighbors ó K Vecinos Más Cercanos.
3. Regresión Logística. Utilizar el método Stochastic Gradient Descent.

De estos tres tipos de modelos, cuál creen que es el más adecuado para nuestro caso de aplicación?

**Elegir el modelo que consideren que mejor aplica a nuestro problema.** Para ello, recuerden que los pasos a seguir en la selección pueden esquematizarse como sigue:

#### 1. Descripción de la Hipótesis

¿Cuál es nuestro problema? ¿Cómo se caracteriza? ¿Cuál es la hipótesis?

#### 2. Selección de Regularizador

 ¿Utilizarán algún regularizador?¿Cuál?

#### 3. Selección de Función de Costo

¿Cuál será la función de costo utilizada?

#### 4. Justificación de las Selecciones

¿Por qué eligieron el modelo, el regularizador y la función de costo previas?

Finalmente, para el modelo selecionado:

- Utilizar el método *Grid Search*, o de búsqueda exahustiva, con *cross-validation* para profundizar en la búsqueda y selección de hiperparámetros.
- Calcular métricas sobre el conjunto de entrenamiento y de evaluación para los mejores parámetros obtenidos:
    + Accuracy o exactitud
    + Reporte de clasificación
    + Confusion matrix o matriz de confusión (graficar como heatmap)
    + Curva ROC y área bajo la curva (AUC).
---

Recuerden que la ciencia de datos es un **proceso circular, continuo y no lineal**. Es decir, si los datos requieren de mayor procesamiento para satisfacer las necesidades de algoritmos de ML (cualesquiera de ellos), vamos a volver a la etapa inicial para, por ejemplo, crear nuevas features, tomar decisiones diferentes sobre valores faltantes o valores atípicos (outliers), descartar features, entre otras.

# Resolución

## I. Análisis  Contenido (Cont)

### 1. Features geográficas

Cargue el Dataset con las variables generadas en el práctico 2.

#### Generación de grafos y calculo de rutas

1 - Usando las técnicas mostradas en el notebook sobre procesamiento de información georeferenciada, construya un grafo basado en la capa de línea de distribición de media tensión:
  - Utilize los puntos de conexión PCON_1 y PCON_2 como nodos, y los segmentos como edges.
  - Adicione COMP y COD_ID como atributos de edge (aristas)
  - Asocie cada transformador untrd con un nodo
  - Asocie cada punto de conexión los circuitos de media tension (CTMT) con la subestación, a un nodo.

2 - Seleccione al menos 5 métricas de grafo, calcule sus valores para los nodos asociados a cada fila y adicionelos como features al dataset.

3 - Calcule la distancia de cada untrd a su correspondiente punto de conexión con la subestación, ponderando por COMP (longitud de segmento), utilize el parametro "method" para calcular diferentes tipos de distancias.
Adicione los resultados como variables del dataset.

4 - Calcule la ruta (secuencia de nodos) hasta la conexión con la subestación, y en base a esta calcule:
 - Resistencia eléctica total de cada conexión
 - Reactancia eléctrica de cada conexión.
 [Opcional]
 - Modulo y ángulo de la Impedancia.
 - Corriente Nonimal media a lo largo de la ruta
 - Corriente Maxima media del conductor a lo largo la ruta

In [4]:
SG = nx.Graph()
SG.add_weighted_edges_from(ssmt[['PN_CON_1','PN_CON_2','COMP']].values.tolist(), weight='length')

In [5]:
ID_attr_dict = {tuple((x[0], x[1])): x[2] for x in ssmt[['PN_CON_1','PN_CON_2','COD_ID']].values}
nx.set_edge_attributes(SG, ID_attr_dict, 'SEG_ID')

In [6]:
ucbt_untrd_untrs.UNI_TR_D = ucbt_untrd_untrs.UNI_TR_D.astype(str)
ucbt_untrd_untrs_with_metrics = pd.merge(untrd[['COD_ID', 'geometry', 'CTMT']], ucbt_untrd_untrs, how='inner', on=None,
                                                left_on=['COD_ID'],
                                                right_on=['UNI_TR_D'], sort=False,
                                                copy=True, indicator=False,
                                                validate=None)

In [7]:
untrd_loc_array = [x.coords[0] for x in ucbt_untrd_untrs_with_metrics.geometry]
#Cargamos los postes y con el algoritmo asociamos cada untrd a cada poste inicial como NODE y ENDNODE
ponnot = gpd.read_file(data_path, driver='FileGDB', layer='PONNOT')
#ponnot = ponnot.to_crs(epsg=31984)
ponnot = ponnot[ponnot.COD_ID.isin(ssmt.PN_CON_1.tolist()+ssmt.PN_CON_2.tolist())] # solamente consideramos los ponnot donde estan conectados los segmentos de MT (ssdmt)
ponnot_loc_array = [x.coords[0] for x in ponnot.geometry]

pn_tree = cKDTree(ponnot_loc_array)
pnd, pni = pn_tree.query(untrd_loc_array, n_jobs=-1)

In [8]:
ucbt_untrd_untrs_with_metrics['NODE'] = [ponnot.iloc[i].COD_ID for i in pni]

In [9]:
ctmt_endpoints = pd.read_csv('./Dataset/endpoint_utf.csv', sep =";")

In [10]:
ctmt_endpoints = pd.read_csv('./Dataset/endpoint_utf.csv', sep =";")
ep_dict = {str(row.CTMT): str(row.PONNOT) for i,row in ctmt_endpoints.iterrows()}
ucbt_untrd_untrs_with_metrics['ENDNODE'] = ucbt_untrd_untrs_with_metrics.CTMT.map(ep_dict)

In [11]:
print('calculando degree')
g_degree = SG.degree(ucbt_untrd_untrs_with_metrics.NODE)

print('calculando neighbor degree')
g_avg_ndeg = nx.average_neighbor_degree(SG)

print('calculando pagerank')
g_pr = nx.pagerank(SG)

calculando degree
calculando neighbor degree
calculando pagerank


In [12]:
# Adicionar al grafico como Atributo de los nodos
nx.set_node_attributes(SG, g_degree, 'degree')
nx.set_node_attributes(SG, g_avg_ndeg, 'avg_nei_degree')
nx.set_node_attributes(SG, g_pr, 'pagerank')

In [13]:
# Mapear valores a al dataframe de transformadores
ucbt_untrd_untrs_with_metrics['g_degree'] = ucbt_untrd_untrs_with_metrics.NODE.map(g_degree)
ucbt_untrd_untrs_with_metrics['g_avg_ndeg'] = ucbt_untrd_untrs_with_metrics.NODE.map(g_avg_ndeg)
ucbt_untrd_untrs_with_metrics['g_pagerank'] = ucbt_untrd_untrs_with_metrics.NODE.map(g_pr)

In [14]:
nx.shortest_path_length(SG, ucbt_untrd_untrs_with_metrics.NODE[0], ucbt_untrd_untrs_with_metrics.ENDNODE[0], weight='length')

16177.061999999993

In [15]:
ucbt_untrd_untrs_with_metrics['dist_to_trs'] = [nx.shortest_path_length(SG, trd.NODE, trd.ENDNODE, weight='length') for i, trd in ucbt_untrd_untrs_with_metrics.iterrows()]

In [16]:
ucbt_untrd_untrs_with_metrics['dist_to_trs']

0       16177.062
1       16246.083
2       16821.404
3       17493.381
4       14907.411
          ...    
3329    18705.213
3330    18538.201
3331    17748.824
3332    18833.131
3333    18043.850
Name: dist_to_trs, Length: 3334, dtype: float64

In [17]:
untrd_node_paths = [nx.shortest_path(SG, trd.NODE, trd.ENDNODE) for i, trd in ucbt_untrd_untrs_with_metrics.iterrows()]

In [18]:
# Carguemos la tabla de consumidores de baja tension denominada UCBT
cond = gpd.read_file(data_path, driver='FileGDB', layer='SEGCON')
cond.set_index('COD_ID', inplace=True)
cond.shape, cond.columns

((90, 39),
 Index(['DIST', 'GEOM_CAB', 'FORM_CAB', 'MAT_FAS_1', 'MAT_FAS_2', 'MAT_FAS_3',
        'MAT_NEU', 'ISO_FAS_1', 'ISO_FAS_2', 'ISO_FAS_3', 'ISO_NEU', 'CND_FAS',
        'R1', 'X1', 'FTRCNV', 'CNOM', 'CMAX', 'CM_FAS', 'TUC_FAS', 'A1_FAS',
        'A2_FAS', 'A3_FAS', 'A4_FAS', 'A5_FAS', 'A6_FAS', 'CM_NEU', 'TUC_NEU',
        'A1_NEU', 'A2_NEU', 'A3_NEU', 'A4_NEU', 'A5_NEU', 'A6_NEU', 'DESCR',
        'BIT_FAS_1', 'BIT_FAS_2', 'BIT_FAS_3', 'BIT_NEU', 'geometry'],
       dtype='object'))

In [19]:
cond= cond[['R1','X1']]

In [20]:
untrd_seg_paths = [[SG.edges[path[i-1:i+1]]['SEG_ID'] for i in range(1,len(path))] for path in untrd_node_paths]

In [21]:
ssmt.set_index('COD_ID', inplace=True)

In [22]:
ssmt = ssmt.join(cond, on='TIP_CND', how='left')
ssmt['R'] = ssmt.R1 * ssmt.COMP / 1000
ssmt['X'] = ssmt.X1 * ssmt.COMP / 1000


In [23]:
untrd_ele = pd.DataFrame([ssmt[['COMP','R','X']].loc[r].sum().to_list() for r in untrd_seg_paths], index=ucbt_untrd_untrs_with_metrics.index, columns=['COMP','R','X'])

In [24]:

ucbt_untrd_untrs_with_metrics[['COMP','R','X']] = untrd_ele

ucbt_untrd_untrs_with_metrics['impedancia'] = ((ucbt_untrd_untrs_with_metrics['R'])**2 + (ucbt_untrd_untrs_with_metrics['X'])**2)**(1/2)
ucbt_untrd_untrs_with_metrics['angulo_impedancia'] = np.arctan(ucbt_untrd_untrs_with_metrics['X'] / ucbt_untrd_untrs_with_metrics['R'])
ucbt_untrd_untrs_with_metrics

,COD_ID,geometry,CTMT,UNI_TR_D,GRU_TAR,DIC,FIC,ENE_MED,ENE_MAX,UNI_TR_S,...,ENDNODE,g_degree,g_avg_ndeg,g_pagerank,dist_to_trs,COMP,R,X,impedancia,angulo_impedancia
0,26877970,POINT (-42.59528 -22.19312),27117490,26877970,1,14.950000,7.00,189.000000,274.0,27707625,...,27707667,1,4.0,0.000022,16177.062,16211.275,10.679279,6.491497,12.497461,0.546178
1,26877971,POINT (-42.59618 -22.19249),27117490,26877971,1,49.560000,11.00,112.166667,136.0,27707625,...,27707667,1,4.0,0.000022,16246.083,16280.296,10.786330,6.522350,12.604998,0.543858
2,26877972,POINT (-42.59800 -22.19074),27117490,26877972,1,43.265000,10.25,59.354167,519.0,27707625,...,27707667,2,2.0,0.000040,16821.404,16855.617,11.678653,6.779518,13.503807,0.525962
3,26877973,POINT (-42.60002 -22.18566),27117490,26877973,1,25.036667,8.00,92.444444,343.0,27707625,...,27707667,1,3.0,0.000023,17493.381,17527.594,12.720890,7.079892,14.558362,0.507863
4,26877977,POINT (-42.60092 -22.21269),27117490,26877977,1,62.680000,10.00,37.708333,130.0,27707625,...,27707667,1,2.0,0.000026,14907.411,14941.624,7.934969,5.943998,9.914375,0.642918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3329,26876494,POINT (-42.32841 -22.37498),27117483,26876494,1,62.690000,15.00,31.166667,48.0,27707534,...,27707554,2,2.0,0.000041,18705.213,18705.213,7.987270,7.195785,10.750619,0.733316
3330,26876495,POINT (-42.31524 -22.31688),27117483,26876495,1,16.150000,6.00,86.416667,164.0,27707534,...,27707554,2,2.0,0.000044,18538.201,18538.201,8.357406,7.175402,11.015109,0.709447
3331,26876496,POINT (-42.30362 -22.31482),27117483,26876496,1,16.150000,6.00,80.916667,89.0,27707534,...,27707554,1,3.0,0.000023,17748.824,17748.824,7.808364,6.843330,10.382761,0.719628
3332,26876497,POINT (-42.30930 -22.37747),27117483,26876497,1,19.580000,11.00,264.500000,343.0,27707534,...,27707554,1,2.0,0.000025,18833.131,18833.131,9.342956,7.367089,11.898102,0.667700


## II. Preprocesamiento


### 2. Normalización de Atributos

Aplicar al dataset la normalización de atributos que consideren adecuada.

In [25]:
# Pueden utilizar los siguientes métodos, por ejemplo:

min_max_scaler = preprocessing.MinMaxScaler()
standard_scaler = preprocessing.StandardScaler()


### 3. Mezca Aleatória y División en Train/Test

Primeramente, deberán mezclar los datos aleatoriamente. Luego, para dividir en Train/Test el dataset, aplicar el split utilizando un 20% de datos para este último.

En este punto, deberán obtener cuatro conjuntos de datos, para ambos datasets: ```X_train```, ```X_test```, ```y_train``` y ```y_test```.

## III. Aplicación de Modelos de Clasificación

### 1. Perceptron
A continuación se aplicará un clasificador Perceptrón.


In [26]:
ucbt_untrd_untrs_with_metrics.columns


Index(['COD_ID', 'geometry', 'CTMT', 'UNI_TR_D', 'GRU_TAR', 'DIC', 'FIC',
       'ENE_MED', 'ENE_MAX', 'UNI_TR_S', 'ARE_LOC', 'geometry_untrd',
       'geometry_untrs', 'EUC_DIST_UNTRS', 'EUC_DIST_CENT', 'EUC_DIST_PRIMARY',
       'EUC_DIST_SECONDARY', 'TIEMPO_MIN', 'DISTANCIA_KM', 'NODE', 'ENDNODE',
       'g_degree', 'g_avg_ndeg', 'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X',
       'impedancia', 'angulo_impedancia'],
      dtype='object')

In [27]:
ucbt_untrd_untrs_with_metrics = pd.get_dummies(ucbt_untrd_untrs_with_metrics, columns=['GRU_TAR'], drop_first=True)


In [28]:
ucbt_untrd_untrs_with_metrics['ARE_LOC'].unique()

array([0, 1], dtype=int64)

In [29]:
ucbt_untrd_untrs_with_metrics[['ENE_MED', 'ENE_MAX', 'R', 'X', 'impedancia', 'EUC_DIST_UNTRS',
                               'EUC_DIST_CENT', 'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY',
                               'TIEMPO_MIN', 'DISTANCIA_KM',
                               'g_degree', 'g_avg_ndeg',
                               'g_pagerank', 'dist_to_trs', 'COMP']] = min_max_scaler.fit_transform(ucbt_untrd_untrs_with_metrics[['ENE_MED', 'ENE_MAX', 'R', 'X', 'impedancia', 'EUC_DIST_UNTRS',
                               'EUC_DIST_CENT', 'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
                               'TIEMPO_MIN', 'DISTANCIA_KM', 
                               'g_degree', 'g_avg_ndeg',
                               'g_pagerank', 'dist_to_trs', 'COMP']])
ucbt_untrd_untrs_with_metrics.describe()

,DIC,FIC,ENE_MED,ENE_MAX,UNI_TR_S,ARE_LOC,EUC_DIST_UNTRS,EUC_DIST_CENT,EUC_DIST_PRIMARY,EUC_DIST_SECONDARY,...,g_avg_ndeg,g_pagerank,dist_to_trs,COMP,R,X,impedancia,angulo_impedancia,GRU_TAR_2,GRU_TAR_3
count,3334.000000,3334.000000,3334.000000,3334.000000,3.334000e+03,3334.000000,3334.000000,3334.000000,3334.000000,3334.000000,...,3334.000000,3334.000000,3334.000000,3334.000000,3334.000000,3334.000000,3334.000000,3334.000000,3334.000000,3334.000000
mean,18.575886,7.593354,0.367569,0.095926,3.245994e+07,0.554889,0.260111,0.320440,0.200438,0.170131,...,0.199110,0.259652,0.261482,0.254407,0.192423,0.242545,0.209833,0.755220,0.049790,0.035993
std,22.964956,6.608679,0.160641,0.131142,1.508740e+07,0.497053,0.189484,0.231923,0.253354,0.169011,...,0.115436,0.143839,0.218651,0.212894,0.198185,0.207894,0.198534,0.179550,0.217544,0.186300
min,0.000000,0.000000,0.000000,0.000000,2.770753e+07,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.352911,0.000000,0.000000
25%,2.442833,2.200000,0.286764,0.006355,2.770753e+07,0.000000,0.102763,0.136958,0.012974,0.040078,...,0.142857,0.100438,0.085538,0.083296,0.043699,0.076659,0.056696,0.616641,0.000000,0.000000
50%,9.520000,6.000000,0.374506,0.028123,2.770775e+07,1.000000,0.232410,0.266189,0.077176,0.121979,...,0.142857,0.337429,0.199608,0.196217,0.119408,0.184758,0.145195,0.742856,0.000000,0.000000
75%,25.827500,11.000000,0.439146,0.148716,2.770786e+07,1.000000,0.372916,0.460273,0.306615,0.255093,...,0.285714,0.375304,0.389537,0.376872,0.288506,0.351554,0.307712,0.886667,0.000000,0.000000
max,163.100000,44.500000,1.000000,1.000000,8.034524e+07,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.149996,1.000000,1.000000


In [30]:
#Agregamos una nueva columna llamada DIC_above que representa un 1 o un 0 en caso de que dicha fila, la DIC sea mayor al promedio será 1, de lo contrario 0.
ucbt_untrd_untrs_with_metrics['DIC_above'] = np.where(ucbt_untrd_untrs_with_metrics['DIC'] > ucbt_untrd_untrs_with_metrics['DIC'].mean(), 1, 0)
#Agregamos una nueva columna llamada FIC_above que representa un 1 o un 0 en caso de que dicha fila, la FIC sea mayor al promedio será 1, de lo contrario 0.
ucbt_untrd_untrs_with_metrics['FIC_above'] = np.where(ucbt_untrd_untrs_with_metrics['FIC'] > ucbt_untrd_untrs_with_metrics['FIC'].mean(), 1, 0)


In [31]:
ucbt_untrd_untrs_with_metrics.columns

Index(['COD_ID', 'geometry', 'CTMT', 'UNI_TR_D', 'DIC', 'FIC', 'ENE_MED',
       'ENE_MAX', 'UNI_TR_S', 'ARE_LOC', 'geometry_untrd', 'geometry_untrs',
       'EUC_DIST_UNTRS', 'EUC_DIST_CENT', 'EUC_DIST_PRIMARY',
       'EUC_DIST_SECONDARY', 'TIEMPO_MIN', 'DISTANCIA_KM', 'NODE', 'ENDNODE',
       'g_degree', 'g_avg_ndeg', 'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X',
       'impedancia', 'angulo_impedancia', 'GRU_TAR_2', 'GRU_TAR_3',
       'DIC_above', 'FIC_above'],
      dtype='object')

In [32]:
# Para dividir el dataset, utilizar el siguiente módulo:

_ds_shuff = shuffle(ucbt_untrd_untrs_with_metrics)

# Y luego el módulo:
X = _ds_shuff[['ARE_LOC', 'GRU_TAR_2', 'GRU_TAR_3', 'ENE_MED', 
               'ENE_MAX', 'EUC_DIST_UNTRS',
               'EUC_DIST_CENT', 'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
               'TIEMPO_MIN', 'DISTANCIA_KM',
               'g_degree', 'g_avg_ndeg','g_pagerank', 'dist_to_trs', 
               'COMP', 'R', 'X', 'impedancia',
               'angulo_impedancia']]

y = _ds_shuff['DIC_above']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Notar que X e y son np.arrays. Además, pueden usar el parámetro que incluye train_test_split para mezclar.

In [33]:
# En principio, pueden utilizar el módulo que sigue, con los parámetros por defecto y los que definan a continuación:

#penalty = 0.001  
alpha = 0.
max_iter = 5
#tol

model = Perceptron(alpha = alpha, fit_intercept=True, max_iter = max_iter, shuffle=True, random_state=0, class_weight=None, warm_start=False)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc_perceptron = recall_score(y_test, y_pred, average='macro')

print("Precisión de las clasificaciones del perceptron para DIC: ", acc_perceptron)

Precisión de las clasificaciones del perceptron para DIC:  0.7583864763640045


D:\Anaconda3\envs\diplo\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [34]:
#Corremos el perceptron para la FIC
y = _ds_shuff['FIC_above']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#penalty = 0.001  
alpha = 0.
max_iter = 5
#tol

model = Perceptron(alpha = alpha, fit_intercept=True, max_iter = max_iter, shuffle=True, random_state=0, class_weight=None, warm_start=False)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc_perceptron = recall_score(y_test, y_pred, average='macro')

print("Precisión de las clasificaciones del perceptron para FIC: ", acc_perceptron)

Precisión de las clasificaciones del perceptron para FIC:  0.7981994589047485


D:\Anaconda3\envs\diplo\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


### 2. K-NN

A continuación se aplicará un clasificador K-NN.

In [35]:
metric = 'manhattan'

X = _ds_shuff[['ARE_LOC', 'GRU_TAR_2', 'GRU_TAR_3', 'ENE_MED', 'ENE_MAX', 
               'EUC_DIST_UNTRS', 'EUC_DIST_CENT',
               'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
               'TIEMPO_MIN', 'DISTANCIA_KM',
               'g_degree', 'g_avg_ndeg',
       'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X', 'impedancia',
       'angulo_impedancia']]

y = _ds_shuff['DIC_above']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

for i in range(1, 20):
    model = KNeighborsClassifier(n_neighbors=i, metric=metric)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_knn = recall_score(y_test, y_pred, average='macro')
    print("Para " + str(i) + " vecinos, tenemos: %.2f" % acc_knn + " de precisión.")

Para 1 vecinos, tenemos: 0.79 de precisión.
Para 2 vecinos, tenemos: 0.76 de precisión.
Para 3 vecinos, tenemos: 0.80 de precisión.
Para 4 vecinos, tenemos: 0.77 de precisión.
Para 5 vecinos, tenemos: 0.81 de precisión.
Para 6 vecinos, tenemos: 0.78 de precisión.
Para 7 vecinos, tenemos: 0.81 de precisión.
Para 8 vecinos, tenemos: 0.78 de precisión.
Para 9 vecinos, tenemos: 0.80 de precisión.
Para 10 vecinos, tenemos: 0.78 de precisión.
Para 11 vecinos, tenemos: 0.80 de precisión.
Para 12 vecinos, tenemos: 0.79 de precisión.
Para 13 vecinos, tenemos: 0.80 de precisión.
Para 14 vecinos, tenemos: 0.77 de precisión.
Para 15 vecinos, tenemos: 0.79 de precisión.
Para 16 vecinos, tenemos: 0.77 de precisión.
Para 17 vecinos, tenemos: 0.79 de precisión.
Para 18 vecinos, tenemos: 0.77 de precisión.
Para 19 vecinos, tenemos: 0.79 de precisión.


In [36]:
metric = 'manhattan'

X = _ds_shuff[['ARE_LOC', 'GRU_TAR_2', 'GRU_TAR_3', 'ENE_MED', 'ENE_MAX', 
               'EUC_DIST_UNTRS', 'EUC_DIST_CENT',
               'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
               'TIEMPO_MIN', 'DISTANCIA_KM',
               'g_degree', 'g_avg_ndeg',
       'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X', 'impedancia',
       'angulo_impedancia']]

y = _ds_shuff['FIC_above']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

for i in range(1, 20):
    model = KNeighborsClassifier(n_neighbors=i, metric=metric)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc_knn = recall_score(y_test, y_pred, average='macro')
    print("Para " + str(i) + " vecinos, tenemos: %.2f" % acc_knn + " de precisión.")

Para 1 vecinos, tenemos: 0.84 de precisión.
Para 2 vecinos, tenemos: 0.83 de precisión.
Para 3 vecinos, tenemos: 0.84 de precisión.
Para 4 vecinos, tenemos: 0.84 de precisión.
Para 5 vecinos, tenemos: 0.85 de precisión.
Para 6 vecinos, tenemos: 0.85 de precisión.
Para 7 vecinos, tenemos: 0.87 de precisión.
Para 8 vecinos, tenemos: 0.84 de precisión.
Para 9 vecinos, tenemos: 0.86 de precisión.
Para 10 vecinos, tenemos: 0.84 de precisión.
Para 11 vecinos, tenemos: 0.85 de precisión.
Para 12 vecinos, tenemos: 0.84 de precisión.
Para 13 vecinos, tenemos: 0.85 de precisión.
Para 14 vecinos, tenemos: 0.83 de precisión.
Para 15 vecinos, tenemos: 0.84 de precisión.
Para 16 vecinos, tenemos: 0.82 de precisión.
Para 17 vecinos, tenemos: 0.84 de precisión.
Para 18 vecinos, tenemos: 0.82 de precisión.
Para 19 vecinos, tenemos: 0.83 de precisión.


### 3. Regresión Logística

In [37]:
#penalty =   # TODO: Tipo de regularización: l1 (valor absoluto), l2 (cuadrados).
#alpha =   # TODO: Parámetro de regularización. También denominado como parámetro `lambda`. Debe ser mayor que 0.
max_iter = 10  # TODO: Cantidad máxima de iteraciones del algoritmo.
#tol =   # TODO: Precisión del algoritmo (error mínimo entre una iteración y la siguiente).

_ds_shuff = shuffle(ucbt_untrd_untrs_with_metrics)

# Y luego el módulo:
X = _ds_shuff[['ARE_LOC', 'GRU_TAR_2', 'GRU_TAR_3', 'ENE_MED', 'ENE_MAX',
               'EUC_DIST_UNTRS', 'EUC_DIST_CENT',
               'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
               'TIEMPO_MIN', 'DISTANCIA_KM',
               'g_degree', 'g_avg_ndeg',
       'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X', 'impedancia',
       'angulo_impedancia']]

y = _ds_shuff['DIC_above']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)



model = LogisticRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
acc_logistic_regression = recall_score(y_test, y_pred, average='macro')

print("Precisión la regresión logística para DIC: ", acc_logistic_regression)

Precisión la regresión logística para DIC:  0.7458176307551195


In [38]:
#penalty =   # TODO: Tipo de regularización: l1 (valor absoluto), l2 (cuadrados).
#alpha =   # TODO: Parámetro de regularización. También denominado como parámetro `lambda`. Debe ser mayor que 0.
max_iter = 10  # TODO: Cantidad máxima de iteraciones del algoritmo.
#tol =   # TODO: Precisión del algoritmo (error mínimo entre una iteración y la siguiente).

_ds_shuff = shuffle(ucbt_untrd_untrs_with_metrics)

# Y luego el módulo:
X = _ds_shuff[['ARE_LOC', 'GRU_TAR_2', 'GRU_TAR_3', 'ENE_MED', 'ENE_MAX',
               'EUC_DIST_UNTRS', 'EUC_DIST_CENT',
               'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
               'g_degree', 'g_avg_ndeg',
       'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X', 'impedancia',
       'angulo_impedancia']]

y = _ds_shuff['FIC_above']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)



model = LogisticRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
acc_logistic_regression = recall_score(y_test, y_pred, average='macro')

print("Precisión la regresión logística para FIC: ", acc_logistic_regression)

Precisión la regresión logística para FIC:  0.738313457008245


¿Cómo sería utilizando el método Stochastic Gradient Descent

In [39]:
_ds_shuff = shuffle(ucbt_untrd_untrs_with_metrics)

# Y luego el módulo:
X = _ds_shuff[['ARE_LOC', 'GRU_TAR_2', 'GRU_TAR_3', 'ENE_MED', 'ENE_MAX',
               'EUC_DIST_UNTRS', 'EUC_DIST_CENT',
               'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
               'TIEMPO_MIN', 'DISTANCIA_KM',
               'g_degree', 'g_avg_ndeg',
       'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X', 'impedancia',
       'angulo_impedancia']]

y = _ds_shuff['DIC_above']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)



model = SGDClassifier()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
acc_SGDClassifier_dic = recall_score(y_test, y_pred, average='macro')

print("Precisión la regresión logística para DIC: ", acc_SGDClassifier_dic)

Precisión la regresión logística para DIC:  0.7538055055738248


In [40]:
_ds_shuff = shuffle(ucbt_untrd_untrs_with_metrics)

# Y luego el módulo:
X = _ds_shuff[['ARE_LOC', 'GRU_TAR_2', 'GRU_TAR_3', 'ENE_MED', 'ENE_MAX',
               'EUC_DIST_UNTRS', 'EUC_DIST_CENT',
               'EUC_DIST_PRIMARY', 'EUC_DIST_SECONDARY', 
               'TIEMPO_MIN', 'DISTANCIA_KM',
               'g_degree', 'g_avg_ndeg',
       'g_pagerank', 'dist_to_trs', 'COMP', 'R', 'X', 'impedancia',
       'angulo_impedancia']]

y = _ds_shuff['FIC_above']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)



model = SGDClassifier()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
acc_SGDClassifier_fic = recall_score(y_test, y_pred, average='macro')

print("Precisión la regresión logística para FIC: ", acc_SGDClassifier_fic)

Precisión la regresión logística para FIC:  0.8026113813319524


### 4. Selección del Modelo

#### 4.1. Selección y Descripción de Hipótesis

Describir el problema y la hipótesis del modelo.

#### 4.2. Selección de Regularizador

 ¿Utilizarán algún regularizador?¿Cuál?

#### 4.3. Selección de Función de Costo

¿Cuál será la función de costo utilizada?

#### 4.4. Justificación de las Selecciones

A continuación, se justifican las elecciones previas.

### 5. Selección de Parámetros y Métricas Sobre el Conjunto de Evaluación

Para la selección de hiperparámetros, pueden utilizar GridSearch. Además, deben calcular las métricas solicitadas.

In [44]:
# Para la búsqueda de los mejores parámetros, por ejemplo de logistic regression, pueden usar:
exploring_params = {
        'C': [0.5, 1, 2, 5, 10, 20, 100, 200], # Inversa del coeficiente de regularización
        'max_iter': [1000, 5000, 10000],  # Cantidad de iteraciones
        'tol': [0.005, 0.002, 0.001, 0.0001]  # Precisión del algoritmo
    }

m = LogisticRegression()
n_cross_val =   5
scoring = 'roc_auc'
model = GridSearchCV(m, exploring_params, cv=n_cross_val, scoring=scoring)
    
model.fit(X_train, y_train)

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")
print()
print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']
print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
y_true, y_pred = y_test, model.predict(X_test)
print(classification_report(y_true, y_pred), end="\n\n")

print("================================================", end="\n\n")

Mejor conjunto de parámetros:
{'C': 200, 'max_iter': 1000, 'tol': 0.005}


Puntajes de la grilla:

Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       389
           1       0.79      0.76      0.77       278

    accuracy                           0.81       667
   macro avg       0.81      0.80      0.81       667
weighted avg       0.81      0.81      0.81       667





In [42]:
# Para la búsqueda de los mejores parámetros, por ejemplo de K-NN, pueden usar:
exploring_params = {
        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19 ,20], # Cantidad de vecinos.
        'metric': ['cosine', 'euclidean', 'manhattan']  # Metricas
    }

m = KNeighborsClassifier()
n_cross_val =   5
scoring = 'roc_auc'
model = GridSearchCV(m, exploring_params, cv=n_cross_val, scoring=scoring)
    
model.fit(X_train, y_train)

print("Mejor conjunto de parámetros:")
print(model.best_params_, end="\n\n")
print()
print("Puntajes de la grilla:", end="\n\n")
means = model.cv_results_['mean_test_score']
stds = model.cv_results_['std_test_score']

print("Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):", end="\n\n")
y_true, y_pred = y_test, model.predict(X_test)
print(classification_report(y_true, y_pred), end="\n\n")

print("================================================", end="\n\n")

Mejor conjunto de parámetros:
{'metric': 'manhattan', 'n_neighbors': 8}


Puntajes de la grilla:

Reporte de clasificación para el mejor clasificador (sobre conjunto de evaluación):

              precision    recall  f1-score   support

           0       0.86      0.92      0.89       389
           1       0.88      0.79      0.83       278

    accuracy                           0.87       667
   macro avg       0.87      0.86      0.86       667
weighted avg       0.87      0.87      0.87       667





In [43]:
ucbt_untrd_untrs_with_metrics.to_csv("./Dataset/ucbt_untrd_untrs_with_metrics_dist_v2_2.csv")